In [1]:
def get_rolling_returns(PLAN_ID, TRANSACTION_DATE, ROLLING_YEARS):
    query = db_session.query(NAV.NAV_Date, NAV.NAV).filter(NAV.Is_Deleted != 1)\
    .filter(NAV.Plan_Id == PLAN_ID).filter(NAV.NAV_Date <= TRANSACTION_DATE)\
    .filter(NAV.NAV_Type == 'P').order_by(NAV.NAV_Date).all()
    df = pd.DataFrame(query)
    df['NAV_Date'] = df['NAV_Date'].apply(pd.to_datetime)
    df = df.set_index('NAV_Date')
    df["old_NAV"] = df.shift(periods= 365*ROLLING_YEARS, freq='D')["NAV"]
    df["returns"] = ((df["NAV"] - df["old_NAV"])/ df["old_NAV"])*100
    df["returns"].median()
    df["returns"].max()
    df["returns"].min()
    df["returns"].mean()
    total = len(df)
    ret = df['returns']
    sum((ret>50) & (ret<75))
    return df

In [2]:
from utils.finalyca_store import get_finalyca_scoped_session
from utils.utils import print_query
from fin_models.transaction_models import NAV, RiskFreeIndexRate
from bizlogic.analytics import get_risk_ratios
import datetime
from sqlalchemy import func

In [3]:
db_session = get_finalyca_scoped_session(False)
PLAN_ID = 6
TRANSACTION_DATE = datetime.datetime(2022, 5, 31)
ROLLING_YEARS = 1
INDEX_ID = 242

In [4]:
sql_rate_max_date = db_session.query(func.max(RiskFreeIndexRate.Date)).filter(RiskFreeIndexRate.Date <= TRANSACTION_DATE).scalar()
sql_rate = db_session.query(RiskFreeIndexRate).filter(RiskFreeIndexRate.Date == sql_rate_max_date).one_or_none()
risk_free_index_date = sql_rate.Date
risk_free_index_rate = float(sql_rate.Rate)
get_risk_ratios(db_session, PLAN_ID, INDEX_ID, TRANSACTION_DATE, 36, risk_free_index_rate)

{'StandardDeviation': 21.637877114147333,
 'SharpeRatio': 0.4451013927898268,
 'Alpha': -0.4032325778579189,
 'R_Squared': 0.8142440476414253,
 'Beta': 0.905264110759205,
 'Mean': 13.536490332122096,
 'Sortino': 0.4622798062021884}